In [1]:
%matplotlib inline

In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [3]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(-90.00,90.00,2000)
lngs = np.random.uniform(-180.00,180.00,2000)
lat_lngs=zip(lats,lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print(coordinates[0])

(-63.62398857847664, -81.5327120233137)


In [5]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

cities=[]
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
print(len(cities))

777


In [6]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
city_url = url + "&q=" + "London"
weather = requests.get(city_url).json()
weather

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 59.52,
  'feels_like': 58.98,
  'temp_min': 54.55,
  'temp_max': 63,
  'pressure': 1022,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 10.36, 'deg': 280},
 'clouds': {'all': 75},
 'dt': 1623468139,
 'sys': {'type': 2,
  'id': 2019646,
  'country': 'GB',
  'sunrise': 1623469398,
  'sunset': 1623529057},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [10]:
#print the beginning of the logging
print("Beginning Data Retrieval")
print("-------------------------")
print("** est run time : 4 min **")
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data=[]

#loop through the cities in the list

for i,city in enumerate(cities):
    city_url = url + "&q=" + city.replace(" ","+")
    
    try:
        city_weather = requests.get(city_url).json()
        
        #parse out the data needed
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        country = city_weather["sys"]["country"]
        max_temp = city_weather["main"]["temp_max"]
        humidity = city_weather["main"]["humidity"]
        cloudiness= city_weather["clouds"]['all']
        windspeed = city_weather["wind"]["speed"]
        city_desc = city_weather["weather"][0]["description"]
        
        city_data.append({"City": city,
                          "Country": country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp" : max_temp,
                          "Humidity" : humidity,
                          "Cloudiness" : cloudiness,
                          "Wind Speed": windspeed,
                          "Current Description": city_desc
                          })
        
    # If an error is experienced, skip the city.
    except:
        continue


# print the number of cities written to the list, and the number not found
rec_count = len(cities)-len(city_data)        
print(f"{len(city_data)} added to the list, {rec_count} skipped")
#Indicate that Data loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")       

Beginning Data Retrieval
-------------------------
** est run time : 4 min **
712 added to the list, 65 skipped
-----------------------------
Data Retrieval Complete      
-----------------------------


In [11]:
#adding the list into a dataframe

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,punta arenas,CL,-53.1500,-70.9167,35.71,93,0,5.75,clear sky
1,carnarvon,AU,-24.8667,113.6333,71.67,57,40,10.36,scattered clouds
2,longyearbyen,SJ,78.2186,15.6401,37.24,65,75,5.75,broken clouds
3,mataura,NZ,-46.1927,168.8643,49.53,80,39,3.09,scattered clouds
4,nikolskoye,RU,59.7035,30.7861,64.40,90,100,5.01,overcast clouds
5,ponta do sol,PT,32.6667,-17.1000,68.63,76,0,5.46,clear sky
6,atuona,PF,-9.8000,-139.0333,79.65,74,38,16.62,light rain
7,esperance,AU,-33.8667,121.9000,60.64,72,100,7.23,moderate rain
8,albany,US,42.6001,-73.9662,64.67,82,100,2.13,overcast clouds
9,coquimbo,CL,-29.9533,-71.3436,51.53,100,90,1.14,fog


In [14]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")